# PROCESO DE EXTRACCIÓN - TRANSFORMACIÓN - LIMPIEZA DE LOS REGISTROS DE ACCIDENTES OCURRIDOS EN LA CIUDAD DE BUENOS AIRES

## 1 - DESCARGA DE LOS DATASETS PARA EL PROCESO...

url = https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales

## 2 - IMPORTACIÓN DE LAS LIBRERIAS PARA EL PROCESO...

In [2]:
import os
import pandas as pd
from datetime import datetime, time
from datetime import datetime, time, timedelta
import numpy as np

## 3 - EXPLORACIÓN INICIAL DE LOS ARCHIVOS CON FORMATO .xlsx Y CREACIÓN DE LOS DATAFRAMES PARA EL PROCESO DE ETL...

In [3]:
# LECTURA DE LOS ARCHIVOS...

# Ruta de la carpeta que contiene los archivos
carpeta_dataset = "DATASET_BRUTO"

# Nombre de los archivos
archivo_homicidios = "homicidios.xlsx"
archivo_lesiones = "lesiones.xlsx"

# Ruta completa de los archivos
ruta_homicidios = os.path.join(carpeta_dataset, archivo_homicidios)
ruta_lesiones = os.path.join(carpeta_dataset, archivo_lesiones)

# Obtener nombres de las hojas en el archivo 'homicidios.xlsx'
hojas_homicidios = pd.ExcelFile(ruta_homicidios).sheet_names

# Obtener nombres de las hojas en el archivo 'lesiones.xlsx'
hojas_lesiones = pd.ExcelFile(ruta_lesiones).sheet_names

# Imprimir en pantalla los nombres de las hojas
print("Hojas en el archivo 'homicidios.xlsx':")
print(hojas_homicidios)

print("\nHojas en el archivo 'lesiones.xlsx':")
print(hojas_lesiones)

Hojas en el archivo 'homicidios.xlsx':
['HECHOS', 'DICCIONARIO_HECHOS', 'VICTIMAS', 'DICCIONARIO_VICTIMAS', 'clas']

Hojas en el archivo 'lesiones.xlsx':
['HECHOS', 'DICCIONARIO_HECHOS', 'VICTIMAS', 'DICCIONARIO_VICTIMAS']


In [4]:
# CREACIÓN DE LOS DATAFRAMES CORRESPONDIENTES AL ARCHIVO "homicidios.xlsx"

# Ruta del archivo
archivo_excel = 'DATASET_BRUTO/homicidios.xlsx'

# Lee la hoja 'HECHOS' del archivo Excel en un DataFrame
try:
    hechos_homicidios = pd.read_excel(archivo_excel, sheet_name='HECHOS')
    diccionario_hechos_homicidios = pd.read_excel(archivo_excel, sheet_name='DICCIONARIO_HECHOS')
    victimas_homicidios = pd.read_excel(archivo_excel, sheet_name='VICTIMAS')
    diccionario_victimas_homicidios = pd.read_excel(archivo_excel, sheet_name='DICCIONARIO_VICTIMAS')
    clas_homicidios = pd.read_excel(archivo_excel, sheet_name='clas')
except Exception as e:
    print(f"Error al leer el archivo: {e}")

print("EL PROCESO EN EL ARCHIVO homicidios.xlsx SE EJECUTÓ CON ÉXITO...")

EL PROCESO EN EL ARCHIVO homicidios.xlsx SE EJECUTÓ CON ÉXITO...


## 4 - PROCESO DE ETL PARA EL ARCHIVO homicidios.xlsx...

En esta parte del proceso se tiene disponible los siguientes DATAFRAMES que resultaron del archivo "homicidios.xlsx". Los dataframes mencionados son:

* hechos_homicidios
* diccionario_hechos_homicidios
* victimas_homicidios
* diccionario_victimas_homicidios
* clas_homicidios

De estos dataframes, solo se utilizará el denominado "hechos_homicidios" y "victimas_homicidios".

### a. DATAFRAME "HECHOS_HOMICIDIOS"...

#### (1) EXPLORACIÓN INICIAL...

In [5]:
hechos_homicidios.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-5847533969,-3468757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-5850877521,-3466977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-5839040293,-3463189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-5846503904,-3468092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-5838718297,-3462246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [6]:
hechos_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [7]:
hechos_homicidios.dtypes

ID                               object
N_VICTIMAS                        int64
FECHA                    datetime64[ns]
AAAA                              int64
MM                                int64
DD                                int64
HORA                             object
HH                               object
LUGAR_DEL_HECHO                  object
TIPO_DE_CALLE                    object
Calle                            object
Altura                          float64
Cruce                            object
Dirección Normalizada            object
COMUNA                            int64
XY (CABA)                        object
pos x                            object
pos y                            object
PARTICIPANTES                    object
VICTIMA                          object
ACUSADO                          object
dtype: object

In [8]:
hechos_homicidios.shape

(696, 21)

In [9]:
hechos_homicidios.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

In [10]:
hechos_homicidios.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

#### (2) PROCESO DE ETL DEL DATAFRAME...

In [11]:
# ELIMINACIÓN DE LAS COLUMNAS INNECESARIAS...

columnas_a_eliminar = ['FECHA', 'HORA', 'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', 'XY (CABA)', 'PARTICIPANTES']
hechos_homicidios = hechos_homicidios.drop(columnas_a_eliminar, axis=1)

In [12]:
# RENOMBRAR LAS COLUMNAS...

nuevos_nombres = {'N_VICTIMAS':'NUMERO_DE_VICTIMAS', 'AAAA': 'AÑO', 'MM': 'MES', 'DD': 'DIA', 'HH': 'FRANJA_HORARIA', 'Altura': 'ALTURA'}
hechos_homicidios = hechos_homicidios.rename(columns=nuevos_nombres)

In [13]:
# NORMALIZACIÓN DE LOS DATOS DE LAS COLUMNAS EN UN SOLO TIPO DE DATOS...

hechos_homicidios['ID'] = hechos_homicidios['ID'].astype(str)
hechos_homicidios['LUGAR_DEL_HECHO'] = hechos_homicidios['LUGAR_DEL_HECHO'].astype(str)
hechos_homicidios['TIPO_DE_CALLE'] = hechos_homicidios['TIPO_DE_CALLE'].astype(str)
hechos_homicidios['VICTIMA'] = hechos_homicidios['VICTIMA'].astype(str)
hechos_homicidios['ACUSADO'] = hechos_homicidios['ACUSADO'].astype(str)

In [14]:
# TRABAJO CON LA COLUMNA "FRANJA_HORARIA"...

hechos_homicidios['FRANJA_HORARIA'] = hechos_homicidios['FRANJA_HORARIA'].replace('SD', 6)

In [15]:
# TRABAJO CON LA COLUMNA "LUGAR_DEL_HECHO"...

hechos_homicidios = hechos_homicidios[hechos_homicidios['LUGAR_DEL_HECHO'] != 'SD']

In [16]:
# TRABAJO CON LA COLUMNA "VICTIMA"...

hechos_homicidios['VICTIMA'] = hechos_homicidios['VICTIMA'].replace('SD', 'MOTO')

In [17]:
# TRABAJO CON LA COLUMNA "ACUSADO"...

hechos_homicidios['ACUSADO'] = hechos_homicidios['ACUSADO'].replace('SD', 'AUTO')

In [18]:
# TRABAJO CON LA COLUMNA "pos x" Y "pos y"

hechos_homicidios['pos x'] = pd.to_numeric(hechos_homicidios['pos x'], errors='coerce')
hechos_homicidios['pos y'] = pd.to_numeric(hechos_homicidios['pos y'], errors='coerce')

columnas_a_verificar = ['pos x', 'pos y']

hechos_homicidios = hechos_homicidios.dropna(subset=columnas_a_verificar)

In [19]:
# TRATAMIENTO DE VALORES DEL DATAFRAME...

# Identificar valores nulos en el DataFrame y contarlos por columna
valores_nulos = hechos_homicidios.isnull().sum()

# Identificar valores faltantes (en blanco) en el DataFrame y contarlos por columna
valores_faltantes = (hechos_homicidios == '').sum()

# Identificar valores iguales a "SD" en el DataFrame y contarlos por columna
valores_SD = (hechos_homicidios == 'SD').sum()

# Identificar valores iguales a "sd" en el DataFrame y contarlos por columna
valores_sd = (hechos_homicidios == 'sd').sum()

# Calcular porcentajes
porcentaje_nulos = (valores_nulos / len(hechos_homicidios)) * 100
porcentaje_faltantes = (valores_faltantes / len(hechos_homicidios)) * 100
porcentaje_SD = (valores_SD / len(hechos_homicidios)) * 100
porcentaje_sd = (valores_sd / len(hechos_homicidios)) * 100

# Crear un DataFrame con los resultados
resultados_datos_homicidios = pd.DataFrame({
    'Valores Nulos': valores_nulos,
    'Porcentaje Nulos': porcentaje_nulos,
    'Valores Faltantes': valores_faltantes,
    'Porcentaje Faltantes': porcentaje_faltantes,
    'Valores "SD"': valores_SD,
    'Porcentaje "SD"': porcentaje_SD,
    'Valores "sd"': valores_sd,
    'Porcentaje "sd"': porcentaje_sd
})

In [20]:
resultados_datos_homicidios

,Valores Nulos,Porcentaje Nulos,Valores Faltantes,Porcentaje Faltantes,"Valores ""SD""","Porcentaje ""SD""","Valores ""sd""","Porcentaje ""sd"""
ID,0,0.0,0,0.0,0,0.0,0,0.0
NUMERO_DE_VICTIMAS,0,0.0,0,0.0,0,0.0,0,0.0
AÑO,0,0.0,0,0.0,0,0.0,0,0.0
MES,0,0.0,0,0.0,0,0.0,0,0.0
DIA,0,0.0,0,0.0,0,0.0,0,0.0
FRANJA_HORARIA,0,0.0,0,0.0,0,0.0,0,0.0
LUGAR_DEL_HECHO,0,0.0,0,0.0,0,0.0,0,0.0
TIPO_DE_CALLE,0,0.0,0,0.0,0,0.0,0,0.0
COMUNA,0,0.0,0,0.0,0,0.0,0,0.0
pos x,0,0.0,0,0.0,0,0.0,0,0.0


### b. EXPLORACIÓN INICIAL DEL DATAFRAME "VICTIMAS_HOMICIDIOS"...

#### (1) EXPLORACIÓN INICIAL...

In [21]:
victimas_homicidios.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [22]:
victimas_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [23]:
victimas_homicidios.dtypes

ID_hecho                       object
FECHA                  datetime64[ns]
AAAA                            int64
MM                              int64
DD                              int64
ROL                            object
VICTIMA                        object
SEXO                           object
EDAD                           object
FECHA_FALLECIMIENTO            object
dtype: object

In [24]:
victimas_homicidios.shape

(717, 10)

In [25]:
victimas_homicidios.columns

Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

#### (2) PROCESO DE ETL DEL DATAFRAME...

In [26]:
# ELIMINACIÓN DE LAS COLUMNAS INNECESARIAS...

columnas_a_eliminar = ['FECHA']
victimas_homicidios = victimas_homicidios.drop(columnas_a_eliminar, axis=1)

In [27]:
# RENOMBRAR LAS COLUMNAS...

nuevos_nombres = {'ID_hecho':'ID_VICTIMAS', 'AAAA': 'AÑO', 'MM': 'MES', 'DD': 'DIA', 'VICTIMA': 'VEHICULO_DE_VICTIMA'}
victimas_homicidios = victimas_homicidios.rename(columns=nuevos_nombres)

In [28]:
# NORMALIZACIÓN DE LOS DATOS DE LAS COLUMNAS EN UN SOLO TIPO DE DATOS...

victimas_homicidios['ROL'] = victimas_homicidios['ROL'].astype(str)
victimas_homicidios['VEHICULO_DE_VICTIMA'] = victimas_homicidios['VEHICULO_DE_VICTIMA'].astype(str)
victimas_homicidios['SEXO'] = victimas_homicidios['SEXO'].astype(str)

In [29]:
# TRABAJO CON LA COLUMNA "ROL"

victimas_homicidios['ROL'] = victimas_homicidios['ROL'].replace('SD', 'CONDUCTOR')

In [30]:
# TRABAJO CON LA COLUMNA "VEHICULO_DE_VICTIMA"

victimas_homicidios['VEHICULO_DE_VICTIMA'] = victimas_homicidios['VEHICULO_DE_VICTIMA'].replace('SD', 'MOTO')

In [31]:
# TRABAJO CON LA COLUMNA "SEXO"

victimas_homicidios['SEXO'] = victimas_homicidios['SEXO'].replace('SD', 'MASCULINO')

In [32]:
# TRABAJO CON LA COLUMNA "EDAD"

victimas_homicidios['EDAD'] = victimas_homicidios['EDAD'].replace('SD', 30)

In [33]:
# TRABAJO CON LA COLUMNA "FECHA_FALLECIMIENTO"

fecha_reemplazo = pd.to_datetime('2017-02-26')
victimas_homicidios['FECHA_FALLECIMIENTO'] = victimas_homicidios['FECHA_FALLECIMIENTO'].replace('SD', fecha_reemplazo)

# Convertir la columna 'FECHA_FALLECIMIENTO' al tipo de dato datetime
victimas_homicidios['FECHA_FALLECIMIENTO'] = pd.to_datetime(victimas_homicidios['FECHA_FALLECIMIENTO'])

# Dividir la columna 'FECHA_FALLECIMIENTO' en columnas de año, mes y día
victimas_homicidios['AÑO_FALLECIMIENTO'] = victimas_homicidios['FECHA_FALLECIMIENTO'].dt.year
victimas_homicidios['MES_FALLECIMIENTO'] = victimas_homicidios['FECHA_FALLECIMIENTO'].dt.month
victimas_homicidios['DÍA_FALLECIMIENTO'] = victimas_homicidios['FECHA_FALLECIMIENTO'].dt.day

# Eliminar la columna innecesaria del dataframe

columnas_a_eliminar = ['FECHA_FALLECIMIENTO']
victimas_homicidios = victimas_homicidios.drop(columnas_a_eliminar, axis=1)

In [34]:
# TRATAMIENTO DE VALORES DEL DATAFRAME...

# Identificar valores nulos en el DataFrame y contarlos por columna
valores_nulos = victimas_homicidios.isnull().sum()

# Identificar valores faltantes (en blanco) en el DataFrame y contarlos por columna
valores_faltantes = (victimas_homicidios == '').sum()

# Identificar valores iguales a "SD" en el DataFrame y contarlos por columna
valores_SD = (victimas_homicidios == 'SD').sum()

# Identificar valores iguales a "sd" en el DataFrame y contarlos por columna
valores_sd = (victimas_homicidios == 'sd').sum()

# Calcular porcentajes
porcentaje_nulos = (valores_nulos / len(victimas_homicidios)) * 100
porcentaje_faltantes = (valores_faltantes / len(victimas_homicidios)) * 100
porcentaje_SD = (valores_SD / len(victimas_homicidios)) * 100
porcentaje_sd = (valores_sd / len(victimas_homicidios)) * 100

# Crear un DataFrame con los resultados
resultados_datos_victimas = pd.DataFrame({
    'Valores Nulos': valores_nulos,
    'Porcentaje Nulos': porcentaje_nulos,
    'Valores Faltantes': valores_faltantes,
    'Porcentaje Faltantes': porcentaje_faltantes,
    'Valores "SD"': valores_SD,
    'Porcentaje "SD"': porcentaje_SD,
    'Valores "sd"': valores_sd,
    'Porcentaje "sd"': porcentaje_sd
})

In [35]:
resultados_datos_victimas

,Valores Nulos,Porcentaje Nulos,Valores Faltantes,Porcentaje Faltantes,"Valores ""SD""","Porcentaje ""SD""","Valores ""sd""","Porcentaje ""sd"""
ID_VICTIMAS,0,0.0,0,0.0,0,0.0,0,0.0
AÑO,0,0.0,0,0.0,0,0.0,0,0.0
MES,0,0.0,0,0.0,0,0.0,0,0.0
DIA,0,0.0,0,0.0,0,0.0,0,0.0
ROL,0,0.0,0,0.0,0,0.0,0,0.0
VEHICULO_DE_VICTIMA,0,0.0,0,0.0,0,0.0,0,0.0
SEXO,0,0.0,0,0.0,0,0.0,0,0.0
EDAD,0,0.0,0,0.0,0,0.0,0,0.0
AÑO_FALLECIMIENTO,0,0.0,0,0.0,0,0.0,0,0.0
MES_FALLECIMIENTO,0,0.0,0,0.0,0,0.0,0,0.0


## 5 - PROCESO DE ALMACENAMIENTO DE LOS DATOS LIMPIOS...

In [36]:
# Especifica la ruta del directorio donde deseas guardar los DataFrames en formato Parquet
carpeta_DATASET_LIMPIO = 'dataset_limpio'

# Verifica si la carpeta ya existe, si no, créala
if not os.path.exists(carpeta_DATASET_LIMPIO):
    os.makedirs(carpeta_DATASET_LIMPIO)

# Guarda los DataFrames en archivos Parquet
hechos_homicidios.to_parquet(os.path.join(carpeta_DATASET_LIMPIO, "hechos_homicidios.parquet"))
victimas_homicidios.to_parquet(os.path.join(carpeta_DATASET_LIMPIO, "victimas_homicidios.parquet"))